### Изменяемые параметры

In [ ]:
begin_term = 'Q11424' # "Центр" графа - стартовая точка
rel_limiter = 50 # Максимальное число связей в графе
AUTH_KEY = ""  # Ключ авторизации Гигачат

### Отправка SPARQL-запроса

In [22]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
from pprint import pprint

endpoint_url = "https://query.wikidata.org/sparql"

# Формирует запрос
def form_begin_query(term, num_of_relations):
  begin_query = f"""
  #Subproperties and parts of film
  #defaultView:Graph
  SELECT DISTINCT ?searched ?searchedLabel ?searchedDesc ?relation ?property ?propertyLabel ?propertyDesc WHERE {{
    
    {{
      BIND(wd:{term} as ?searched)
      ?searched ?relation ?property.
      FILTER(?relation in (wdt:P2579, wdt:P1552, wdt:P527, wdt:P1963))
      
      ?searched rdfs:label ?searchedLabel.
      ?searched schema:description ?searchedDesc.
      
      ?property rdfs:label ?propertyLabel.
      ?property schema:description ?propertyDesc.
    }}
    UNION
    {{
      wd:{term} ?relation1 ?searched.
      FILTER(?relation1 in (wdt:P2579, wdt:P1552, wdt:P527, wdt:P1963))
      
      ?searched ?relation ?property.
      FILTER(?relation in (wdt:P2579, wdt:P1552, wdt:P527, wdt:P1963, wdt:P1659))
      
      ?searched rdfs:label ?searchedLabel.
      ?searched schema:description ?searchedDesc.
      
      ?property rdfs:label ?propertyLabel.
      ?property schema:description ?propertyDesc.
    }}
    
    FILTER((lang(?searchedLabel) = 'ru') && regex(STR(?searchedLabel), "^[а-я]") && !regex(STR(?searchedLabel), "код") && !regex(STR(?searchedLabel), "номер")).  
    FILTER((lang(?searchedDesc) = 'ru')).
    FILTER((lang(?propertyLabel) = 'ru') && regex(STR(?propertyLabel), "^[а-я]") && !regex(STR(?propertyLabel), "код") && !regex(STR(?propertyLabel), "номер")).  
    FILTER((lang(?propertyDesc) = 'ru')).
  }} LIMIT {num_of_relations}
  """
  return begin_query

# Отправляет SPARQL-запрос и принимает ответ
def SPARQL_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

In [23]:
SPARQL_result = SPARQL_query(endpoint_url, form_begin_query(begin_term, rel_limiter))

for result in SPARQL_result["results"]["bindings"]:
    pprint(result)

{'property': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P58'},
 'propertyDesc': {'type': 'literal',
                  'value': 'автор(ы) сценария фильма',
                  'xml:lang': 'ru'},
 'propertyLabel': {'type': 'literal', 'value': 'сценарист', 'xml:lang': 'ru'},
 'relation': {'type': 'uri',
              'value': 'http://www.wikidata.org/prop/direct/P1963'},
 'searched': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q11424'},
 'searchedDesc': {'type': 'literal',
                  'value': 'совокупность движущихся изображений, связанных '
                           'единым сюжетом',
                  'xml:lang': 'ru'},
 'searchedLabel': {'type': 'literal', 'value': 'фильм', 'xml:lang': 'ru'}}
{'property': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P57'},
 'propertyDesc': {'type': 'literal',
                  'value': 'режиссёр(ы) фильма, сериала, пьесы, компьютерной '
                           'игры и др. произведений искусства',
         

### Фильтрация результата SPARQL-запроса

In [24]:
# Чистит ненужную информацию о типах из результата запроса для одной строки
def filter_JSON_line(line):
    filtered_line = {
        'searched': line.get('searched').get('value').split("/")[-1],
        'searchedLabel': line.get('searchedLabel').get('value'),
        'searchedDesc': line.get('searchedDesc').get('value'),

        'relation': line.get('relation').get('value').split("/")[-1],

        'property': line.get('property').get('value').split("/")[-1],
        'propertyLabel': line.get('propertyLabel').get('value'),
        'propertyDesc': line.get('propertyDesc').get('value')
    }
    
    return filtered_line

# Чистит весь ответ
def filter_SPARQL_result(input_json):
    return [filter_JSON_line(line) for line in input_json["results"]["bindings"]]

In [25]:
filtered_list = filter_SPARQL_result(SPARQL_result)
pprint(filtered_list)

[{'property': 'P58',
  'propertyDesc': 'автор(ы) сценария фильма',
  'propertyLabel': 'сценарист',
  'relation': 'P1963',
  'searched': 'Q11424',
  'searchedDesc': 'совокупность движущихся изображений, связанных единым '
                  'сюжетом',
  'searchedLabel': 'фильм'},
 {'property': 'P57',
  'propertyDesc': 'режиссёр(ы) фильма, сериала, пьесы, компьютерной игры и др. '
                  'произведений искусства',
  'propertyLabel': 'режиссёр',
  'relation': 'P1963',
  'searched': 'Q11424',
  'searchedDesc': 'совокупность движущихся изображений, связанных единым '
                  'сюжетом',
  'searchedLabel': 'фильм'},
 {'property': 'P136',
  'propertyDesc': 'жанр произведения или жанр, в котором работает автор',
  'propertyLabel': 'жанр',
  'relation': 'P1963',
  'searched': 'Q11424',
  'searchedDesc': 'совокупность движущихся изображений, связанных единым '
                  'сюжетом',
  'searchedLabel': 'фильм'},
 {'property': 'P170',
  'propertyDesc': 'создатель творческог

In [26]:
len(filtered_list)

50

### Работа с API LLM

In [27]:
import requests
import json
import time
from datetime import datetime

# Авторизационные данные
SCOPE = "GIGACHAT_API_PERS"  # Для физических лиц

# URL для получения токена и API
TOKEN_URL = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
API_URL = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

# Класс для управления токеном
class GigaChatClient:
    def __init__(self, auth_key, scope):
        self.auth_key = auth_key
        self.scope = scope
        self.access_token = None
        self.expires_at = 0  # Время истечения токена (в секундах с эпохи)
    
    # Проверка, истек ли токен
    def is_token_valid(self):
        current_time = int(time.time())
        # Добавляем буфер в 60 секунд, чтобы обновлять токен заранее
        return self.access_token is not None and current_time < (self.expires_at - 60)


    def get_access_token(self):
        if self.is_token_valid():
            return self.access_token

        headers = {
            "Content-Type": "application/x-www-form-urlencoded",
            "Accept": "application/json",
            "Authorization": f"Basic {self.auth_key}",
            "RqUID": "6f0b1291-c7f3-43c6-bb2e-9f3efb2dc98e"
        }
        payload = {
            "scope": self.scope
        }

        response = requests.post(TOKEN_URL, headers=headers, data=payload, verify=True)
        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data["access_token"]
            self.expires_at = token_data["expires_at"] // 1000  # Преобразуем миллисекунды в секунды
            #print(f"Токен обновлен, действителен до: {datetime.fromtimestamp(self.expires_at)}")
            return self.access_token
        else:
            raise Exception(f"Ошибка получения токена: {response.status_code}, {response.text}")

    # Отправка запроса к GigaChat
    def send_request(self, message, model="GigaChat", temperature=1.0, max_tokens=512):
        access_token = self.get_access_token()

        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {access_token}"
        }
        payload = {
            "model": model,
            "messages": [{"role": "user", "content": message}],
            "temperature": temperature,
            "max_tokens": max_tokens,
            "function_call": "auto"
        }

        response = requests.post(API_URL, headers=headers, data=json.dumps(payload), verify=True)
        if response.status_code == 200:
            return response.json()["choices"][0]["message"]["content"]
        else:
            raise Exception(f"Ошибка запроса к GigaChat: {response.status_code}, {response.text}")

In [28]:
client = GigaChatClient(AUTH_KEY, SCOPE)

#### Неиспользованный функионал по преобразованию понятий (Гигачат не смог, pymorphy 2 - перебор)

In [29]:
# Формирует запрос для изменения понятий
def form_label_message(label, desc):
    message = f"""Если "{label}" является словосочетанием в именительном падеже, верни его назад без изменений.
    Верни словосочетание в именительном падеже по определению: "{desc}".
    В ответе должно быть только словосочетание.
    """
    return message

# Формирует список новых названий для понятий из Wikidata
def get_changed_names(json_list):

    result_dict = {}

    for json_str in json_list:
        
        id = json_str.get('property')

        if id in result_dict:
            continue

        name = json_str.get('propertyLabel')
        desc = json_str.get('propertyDesc')
        
        result_dict[id] = client.send_request(form_label_message(name, desc), temperature=0.0)
    return result_dict

### Формирование сообщений для LLM

In [30]:
# Формирует заголовок
def form_title_message(main_term, main_term_desc):
    message = f"""
    Сформируй для десятилетнего ребёнка короткий заголовок понятия "{main_term}" со смыслом "{main_term_desc}". 
    Никак его не выделяй заголовок.
    """
    return message

# Формирует описание понятия
def form_desc_message(main_term, main_term_desc):
    message = f"""
    Сформируй десятилетнего ребёнка абзац текста - подробное объяснение для  для понятия "{main_term}" со смыслом "{main_term_desc}" . 
    Текст не должен явно упоминать никаких других понятий, связанных с "{main_term}".
    """
    return message

# Формирует описание для связи понятий
def form_part_message(main_term, main_term_desc, related_term, related_term_desc):
    message = f"""
    Объясни для десятилетнего ребёнка про связь понятий "{main_term}" ({main_term_desc}) и "{related_term}" ({related_term_desc}) .
    Текст не должен явно упоминать никаких других понятий, связанных с {main_term} или {related_term}.
    Текст должен быть коротким. Рассказывай в первую очередь про {related_term}, не пиши текст близко к {main_term_desc}.
    """
    return message

### Автоматическое формирование страниц на основе графа из SPARQL

In [31]:
# Собирает все связи текущего элемента в графе
def get_all_relatives(json_list, id):
    lines = []
    for json_line in json_list:
        if json_line.get('searched') == id:
            lines.append(json_line)
    return lines

# Собирает информацию для "центрального" элемента графа - точки старта
def select_begin_term_info(json_list, term):
    for json_line in json_list:
        if json_line.get('searched') == term:
            return {'searched': json_line.get('searched'), 'searchedLabel': json_line.get('searchedLabel'), 'searchedDesc': json_line.get('searchedDesc')}

In [32]:
import os

FOLDER = './pages/'
os.makedirs(FOLDER, exist_ok=True)

def make_filename(namestring):
    return namestring.replace(" ", "_")

# Создаёт страницу
def form_page(cur_info, rel_info):
    filename = make_filename(cur_info.get('searchedLabel')) + '.md'
    f = open(FOLDER + filename, 'w')
    f.write('# ' + client.send_request(form_title_message(cur_info.get('searchedLabel'), cur_info.get('searchedDesc'))) + '\n')
    f.write('### ' + client.send_request(form_desc_message(cur_info.get('searchedLabel'), cur_info.get('searchedDesc'))))
    for line in rel_info:
        friend_filename = line.get('propertyLabel').replace(" ", "_")
        f.write('\n## [' + client.send_request(form_title_message(line.get('propertyLabel'), line.get('propertyDesc'))) + f"](./{friend_filename}.md)\n")
        f.write(client.send_request(form_part_message(line.get('searchedLabel'), line.get('searchedDesc'), line.get('propertyLabel'), line.get('propertyDesc'))))
    f.close()

# Рекурсивный обход графа с вызыванием создания страниц
def form_pages_inner(json_list, rel_info, id_set):
    for json_line in rel_info:
        new_cur = {'searched': json_line.get('property'), 'searchedLabel': json_line.get('propertyLabel'), 'searchedDesc': json_line.get('propertyDesc')}
        new_term = new_cur.get('searched')
        if new_term in id_set:
            continue
        new_rel = get_all_relatives(json_list, new_term)
        form_page(new_cur, new_rel)
        id_set.add(new_term)
        form_pages_inner(json_list, new_rel, id_set)

# Общая функция для создания всех страниц
def form_pages(json_list):
    created_pages_id = set()
    cur_info = select_begin_term_info(json_list, begin_term)
    rel_info = get_all_relatives(json_list, begin_term)
    form_page(cur_info, rel_info)
    created_pages_id.add(begin_term)
    form_pages_inner(json_list, rel_info, created_pages_id)

In [33]:
form_pages(filtered_list)

## Автоматическое формирование concepts.json

In [34]:
# Сбор всех понятий
def gather_all_terms(json_list):
    term_dict = dict()
    begin_term_info = select_begin_term_info(json_list, begin_term)
    term_dict[begin_term_info.get('searched')] = {"label": begin_term_info.get('searchedLabel'), "description": begin_term_info.get('searchedDesc'), 'filename': make_filename(begin_term_info.get('searchedLabel'))}
    for line in json_list:
        id = line.get('property')
        if id in term_dict:
            continue
        term_dict[id] = {"label": line.get('propertyLabel'), "description": line.get('propertyDesc')}
    return term_dict

# Создание concepts.json
def make_concepts(json_list):
    f = open('./concepts.json', 'w')
    f.write('[\n')
    
    term_dict = gather_all_terms(json_list)

    keys = list(term_dict.keys())
    
    key = keys[0]
    value = term_dict[key]
    new_dict = {"id": key, "label": value['label'], "description": value['description'], "filename": make_filename(value['label'])+'.md'}
    json.dump(new_dict, f, indent=4, ensure_ascii=False)

    for key in keys[1:]:
        f.write(',\n')
        value = term_dict[key]
        new_dict = {"id": key, "label": value['label'], "description": value['description'], "filename": make_filename(value['label'])+'.md'}
        json.dump(new_dict, f, indent=4, ensure_ascii=False)

    f.write('\n]')
    f.close()

In [35]:
make_concepts(filtered_list)

## Автоматическое формирование графика для отчёта

#### Неиспользуемый функционал (Гитхаб не знает PlantUML)

In [36]:

def generate_plantuml(json_list, filename="report.md"):
    with open(filename, "w", encoding="utf-8") as f:
        f.write("### Отчёт\n")
        f.write("## Онтология\n")
        f.write('```plantuml\n')
        f.write("@startuml\n")
        f.write("left to right direction\n")
        
        entities = set()
        for line in json_list:
            entities.add(line['searchedLabel'])
            entities.add(line['propertyLabel'])
        
        for entity in entities:
            f.write(f'class "{entity}"\n')
        
        for line in json_list:
            source = line['searchedLabel']
            target = line['propertyLabel']
            relation = line['relation']
            f.write(f'"{source}" --> "{target}" : "{relation}"\n')
        
        f.write("@enduml\n")
        f.write('```')

#### Генерация mermaid-графика

In [37]:
def generate_mermaid(json_list, filename="report.md"):
    with open(filename, "w", encoding="utf-8") as f:
        f.write("# Отчёт\n")
        f.write("## Онтология\n")
        
        f.write('```mermaid\n')
        f.write("classDiagram\n")
        f.write("  direction LR\n")
        
        for line in json_list:
            source = make_filename(line['searchedLabel']).replace("(", "-").replace(")", "-")
            target = make_filename(line['propertyLabel']).replace("(", "-").replace(")", "-")
            relation = line['relation']
            f.write(f'  {source} --> {target} : "{relation}"\n')
        f.write('```')

In [38]:
generate_mermaid(filtered_list)